In [1]:
# multivariate multi-headed 1d cnn example
from numpy import array
from numpy import hstack
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import concatenate
import math
from numpy import array
import matplotlib.pyplot as plt
from numpy import hstack
import numpy as np
from keras.layers import Dropout
from keras.layers import GRU
from keras.layers import LSTM
from keras.layers.convolutional import AveragePooling1D 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
import keras.layers
import pandas as pd

#dataları 1d cnn yapısı için uygun boyuta getirir
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
	X, y1,y2 = list(), list(),list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y1,seq_y2 = sequences[i:end_ix, :-2], sequences[end_ix-1, -2],sequences[end_ix-1, -1]
		X.append(seq_x)
		y1.append(seq_y1)
		y2.append(seq_y2)
	return array(X), array(y1),array(y2)



In [2]:
#dosyadaki imu datalarını oku(train dataset)
s1=pd.read_csv('data_inert1.txt',  delimiter= '\s+', index_col=False, header=None)
s2=pd.read_csv('data_inert2.txt', delimiter= '\s+', index_col=False, header=None)
s3=pd.read_csv('data_inert3.txt' , delimiter= '\s+', index_col=False, header=None)
s4=pd.read_csv('data_inert4.txt',  delimiter= '\s+', index_col=False, header=None)
s5=pd.read_csv('data_inert5.txt',  delimiter= '\s+', index_col=False, header=None)
s6=pd.read_csv('data_inert6.txt', delimiter= '\s+', index_col=False, header=None)
s7=pd.read_csv('data_inert7.txt' , delimiter= '\s+', index_col=False, header=None) #test dataset
s8=pd.read_csv('data_inert8.txt',  delimiter= '\s+', index_col=False, header=None)
s9=pd.read_csv('data_inert9.txt',  delimiter= '\s+', index_col=False, header=None)
s10=pd.read_csv('data_inert10.txt', delimiter= '\s+', index_col=False, header=None)
s11=pd.read_csv('data_inert11.txt' , delimiter= '\s+', index_col=False, header=None)
s12=pd.read_csv('data_inert12.txt',  delimiter= '\s+', index_col=False, header=None)
s13=pd.read_csv('data_inert13.txt',  delimiter= '\s+', index_col=False, header=None)

#dataları birlerştir(train dataset)
dat=pd.concat([s1,s2,s3,s4,s5,s6,s8,s9,s10,s11,s12,s13], ignore_index=True)
dattest=pd.concat([s7], ignore_index=True)#test dataset

#datasette gerekli kısmı seçiyoruz.  1,2,3 numaralı kolonlar ax,ay,az.  4,5,6 numaralı kolonlar gx,gy,gz(tarin dataset son hali)
data1=dat[[1]]
data2=dat[[2]]
data3=dat[[3]]
data4=dat[[4]]
data5=dat[[5]]
data6=dat[[6]]
#test datasette gerekli kısmı seçiyoruz.  1,2,3 numaralı kolonlar ax,ay,az.  4,5,6 numaralı kolonlar gx,gy,gz(test dataset son hali)
data1test=dattest[[1]]
data2test=dattest[[2]]
data3test=dattest[[3]]
data4test=dattest[[4]]
data5test=dattest[[5]]
data6test=dattest[[6]]




In [3]:
#train ve test dataları numpy vektöre a çeviriyoruz
in_seq1=data1.to_numpy()
in_seq2=data2.to_numpy()
in_seq3=data3.to_numpy()
in_seq4=data4.to_numpy()
in_seq5=data5.to_numpy()
in_seq6=data6.to_numpy()

in_seq1t=data1test.to_numpy()
in_seq2t=data2test.to_numpy()
in_seq3t=data3test.to_numpy()
in_seq4t=data4test.to_numpy()
in_seq5t=data5test.to_numpy()
in_seq6t=data6test.to_numpy()

#target data oku
q1=pd.read_csv('resp1.csv')
q2=pd.read_csv('resp2.csv')
q3=pd.read_csv('resp3.csv')
q4=pd.read_csv('resp4.csv')
q5=pd.read_csv('resp5.csv')
q6=pd.read_csv('resp6.csv')
q7=pd.read_csv('resp7.csv')
q8=pd.read_csv('resp8.csv')
q9=pd.read_csv('resp9.csv')
q10=pd.read_csv('resp10.csv')
q11=pd.read_csv('resp11.csv')
q12=pd.read_csv('resp12.csv')
q13=pd.read_csv('resp13.csv')
yya=pd.concat([q1,q2,q3,q4,q5,q6,q8,q9,q10,q11,q12,q13], ignore_index=True) # train dataları birleştir
yyatest=pd.concat([q7], ignore_index=True)# test dataları birleştir
yyatest=yyatest[['1','2','9']]#test target gerekli kısmı seçiyoruz.  1,2,9 numaralı kolonlar pos x, pos y,  yaw (test target son hali)

yya=yya[['1','2','9']]#train target gerekli kısmı seçiyoruz.  1,2,9 numaralı kolonlar pos x, pos y,  yaw (train target son hali)

#target ları numpy vektöre çevir
yyx=yya.to_numpy()
yyxtest=yyatest.to_numpy()

#aşağıdaki kod dataları filtreler. bunu kullanmadık sonucu belki daha iyi hale getirir 
'''
from scipy.signal import lfilter
#filter-------------------------------------------------------------------------------------
n = 16  # the larger n is, the smoother curve will be
b = [1.0 / n] * n
a = 1


in_seq1= lfilter(b, a, in_seq1)
in_seq2=lfilter(b, a, in_seq2)
in_seq3=lfilter(b, a, in_seq3)
in_seq4=lfilter(b, a, in_seq4)
in_seq5=lfilter(b, a, in_seq5)
in_seq6=lfilter(b, a, in_seq6)

in_seq1t=lfilter(b, a, in_seq1t)
in_seq2t=lfilter(b, a, in_seq2t)
in_seq3t=lfilter(b, a, in_seq3t)
in_seq4t=lfilter(b, a, in_seq4t)
in_seq5t=lfilter(b, a, in_seq5t)
in_seq6t=lfilter(b, a, in_seq6t)


yyxtest[:,1] = lfilter(b, a, yyxtest[:,1])
yyxtest[:,0] = lfilter(b, a, yyxtest[:,0])
yyxtest[:,2] = lfilter(b, a, yyxtest[:,2])
yyx[:,1] = lfilter(b, a, yyx[:,1])
yyx[:,0] = lfilter(b, a, yyx[:,0])
yyx[:,2] = lfilter(b, a, yyx[:,2])

'''



#train datada deltaposition normunu hesaplar. bunu target olarak kullanacağız
f=yyx.shape
a=np.zeros((f[0], 1))
ttt=0
for i in range(f[0]-1):
    a[i,0]=math.sqrt((yyx[i+1,0]-yyx[i,0])**2+(yyx[i+1,1]-yyx[i,1])**2)
#----------------------------
#f=yyx.shape
#dmag=np.zeros((f[0], 1))
#ttt=0
#for i in range(f[0]-1):
#    dmag[i,0]=a[i+1,0]-a[i,0]

#test datada deltaposition normunu hesaplar. bunu normu tahmin etmek çin target olarak kullanacağız

f=yyxtest.shape
at=np.zeros((f[0], 1))
for i in range(f[0]-1):
    at[i,0]=math.sqrt((yyxtest[i+1,0]-yyxtest[i,0])**2+(yyxtest[i+1,1]-yyxtest[i,1])**2)
#---------------------
#f=yyxtest.shape
#dmagt=np.zeros((f[0], 1))
#ttt=0
#for i in range(f[0]-1):
 #   dmagt[i,0]=at[i+1,0]-at[i,0]


#test datada deltayaw hesaplar. bunu heading i tahmin etmek için target olarak kullanacağız

arr = np.arange(0,len(yyxtest))
f=yyxtest.shape
rrt=np.zeros((f[0], 1))
for i in range(f[0]-1):
    rrt[i,0]=yyxtest[i+1,2]-yyxtest[i,2]


#train datada deltayaw hesaplar. bunu heading i tahmin etmek için target olarak kullanacağız
yyp=yyx.shape
rr=np.zeros((yyp[0], 1))
for i in range(yyp[0]-1):
    rr[i,0]=yyx[i+1,2]-yyx[i,2]




#rr->yaw of train,rrt->yaw of test,at->mag of test,a->mag of train


In [ ]:
#bu kısım oluşturulan target data(pos.norm,heading) matlabdaki çıktıyı veriyormu diye baktık. biraz farklı sonuç verdi. 
arr = np.arange(0,len(yyxtest[:,2]))
sx=np.zeros(len(yyxtest[:,2]))
sy=np.zeros(len(yyxtest[:,2]))

for i in arr:
    sx[i]=at[i]*math.cos(yyxtest[i,2])
    sy[i]=at[i]*math.sin(yyxtest[i,2])

ddd=0
vvv=0
arr = np.arange(0,len(sx))
xft=np.zeros((len(sx),2))


for i in arr:
    xft[i,0]=ddd+sx[i]
    xft[i,1]=vvv+sy[i]
    
    ddd=xft[i,0]
    vvv=xft[i,1]

plt.scatter(xft[:,0],xft[:,1],linewidth=2, linestyle="-", c="r")

plt.scatter(yyxtest[:,1],yyxtest[:,0],linewidth=2, linestyle="-", c="b")

In [5]:



#verileri(test, train target) 1 boyutlu vektör haline getiriyoruz.

out_seq1 = a
out_seq2 = rr
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
in_seq3 = in_seq3.reshape((len(in_seq3), 1))
in_seq4 = in_seq4.reshape((len(in_seq4), 1))
in_seq5 = in_seq5.reshape((len(in_seq5), 1))
in_seq6 = in_seq6.reshape((len(in_seq6), 1))

out_seq1 = out_seq1.reshape((len(out_seq1), 1))
out_seq2 = out_seq2.reshape((len(out_seq2), 1))


In [6]:

out_seqt1 = at
out_seqt2 = rrt
# convert to [rows, columns] structure
in_seq1t = in_seq1t.reshape((len(in_seq1t), 1))
in_seq2t = in_seq2t.reshape((len(in_seq2t), 1))
in_seq3t = in_seq3t.reshape((len(in_seq3t), 1))
in_seq4t = in_seq4t.reshape((len(in_seq4t), 1))
in_seq5t = in_seq5t.reshape((len(in_seq5t), 1))
in_seq6t = in_seq6t.reshape((len(in_seq6t), 1))

out_seqt1 = out_seqt1.reshape((len(out_seqt1), 1))
out_seqt2 = out_seqt2.reshape((len(out_seqt2), 1))




In [7]:
'''
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
scaler.fit(train)
scaled_train = scaler.transform(train)
scaled_test = scaler.transform(test)
true_predictions = scaler.inverse_transform(test_predictions)
'''

'\nfrom sklearn.preprocessing import MinMaxScaler\nscaler=MinMaxScaler()\nscaler.fit(train)\nscaled_train = scaler.transform(train)\nscaled_test = scaler.transform(test)\ntrue_predictions = scaler.inverse_transform(test_predictions)\n'

In [8]:
# oluşturduumuz dikey datayı birleştiriyoruz
dataset = hstack((in_seq1, in_seq2,in_seq3,in_seq4, in_seq5,in_seq6 ,out_seq1,out_seq2))

datasettest = hstack((in_seq1t, in_seq2t,in_seq3t,in_seq4t, in_seq5t,in_seq6t, out_seqt1,out_seqt2))


In [9]:
# choose a number of time steps(pencere boyutu)
n_steps = 48

In [10]:
# test ve train dataları window size ı kullanarak 1d cnn e uygun hale getiriyoruz.
X, y1,y2 = split_sequences(dataset, n_steps)
Xt, yt1,yt2 = split_sequences(datasettest, n_steps)

# one time series per head
n_features = 1
X.shape


(192572, 48, 6)

In [ ]:
'''''
from sklearn.preprocessing import MinMaxScaler
import numpy as np

class MinMaxScaler3D(MinMaxScaler):

    def fit_transform(self, X, y=None):
        x = np.reshape(X, newshape=(X.shape[0]*X.shape[1], X.shape[2]))
        return np.reshape(super().fit_transform(x, y=y), newshape=X.shape)
''''

In [ ]:
#scaler = MinMaxScaler3D()
#X = scaler.fit_transform(X)
#Xt = scaler.fit_transform(Xt)

In [10]:
# separate input data
X1 = X[:, :, 0].reshape(X.shape[0], X.shape[1], n_features)
X2 = X[:, :, 1].reshape(X.shape[0], X.shape[1], n_features)
X3 = X[:, :, 2].reshape(X.shape[0], X.shape[1], n_features)
X4 = X[:, :, 3].reshape(X.shape[0], X.shape[1], n_features)
X5 = X[:, :, 4].reshape(X.shape[0], X.shape[1], n_features)
X6 = X[:, :, 5].reshape(X.shape[0], X.shape[1], n_features)
X1t = Xt[:, :, 0].reshape(Xt.shape[0], Xt.shape[1], n_features)
X2t = Xt[:, :, 1].reshape(Xt.shape[0], Xt.shape[1], n_features)
X3t = Xt[:, :, 2].reshape(Xt.shape[0], Xt.shape[1], n_features)
X4t = Xt[:, :, 3].reshape(Xt.shape[0], Xt.shape[1], n_features)
X5t = Xt[:, :, 4].reshape(Xt.shape[0], Xt.shape[1], n_features)
X6t = Xt[:, :, 5].reshape(Xt.shape[0], Xt.shape[1], n_features)


In [11]:
#1d cnn modeli bunu train ederken çalıştır
'''
inputax = Input(shape=(n_steps, n_features))
inputay = Input(shape=(n_steps, n_features))
inputaz = Input(shape=(n_steps, n_features))
inputwx = Input(shape=(n_steps, n_features))
inputwy = Input(shape=(n_steps, n_features))
inputwz = Input(shape=(n_steps, n_features))
act_func='LeakyReLU'
filter1=1024
filter2=512
k_size1=24
k_size2=16
mp1=8
mp2=4
# first input model
cnn1 = Conv1D(filters=filter1, kernel_size=k_size1, activation=act_func,padding='SAME')(inputax)
cnn1 = MaxPooling1D(pool_size=mp1)(cnn1)
keras.layers.BatchNormalization(),
cnn1=Dropout(0.5)(cnn1)
cnn1 = Conv1D(filters=filter2, kernel_size=k_size2, activation=act_func,padding='SAME')(cnn1)
cnn1 = MaxPooling1D(pool_size=mp2,padding='SAME')(cnn1)
keras.layers.BatchNormalization(),
cnn1=Dropout(0.5)(cnn1)
#cnn1= GRU (32,dropout=0.5,recurrent_dropout=0.5)(cnn1)
cnn1 = Flatten()(cnn1)
cnn1 = Model(inputs=inputax, outputs=cnn1)

# second input model
cnn2 = Conv1D(filters=filter1, kernel_size=k_size1, activation=act_func,padding='SAME')(inputay)
cnn2 = MaxPooling1D(pool_size=mp1)(cnn2)
keras.layers.BatchNormalization(),
cnn2=Dropout(0.5)(cnn2)
cnn2 = Conv1D(filters=filter2, kernel_size=k_size2, activation=act_func,padding='SAME')(cnn2)
cnn2 = MaxPooling1D(pool_size=mp2,padding='SAME')(cnn2)
keras.layers.BatchNormalization(),
cnn2=Dropout(0.5)(cnn2)
#3cnn2= GRU (32,dropout=0.5,recurrent_dropout=0.5)(cnn2)
cnn2 = Flatten()(cnn2)
cnn2 = Model(inputs=inputay, outputs=cnn2)



cnn3 = Conv1D(filters=filter1, kernel_size=k_size1, activation=act_func,padding='SAME')(inputaz)
cnn3 = MaxPooling1D(pool_size=mp1)(cnn3)
keras.layers.BatchNormalization(),
cnn3=Dropout(0.5)(cnn3)
cnn3 = Conv1D(filters=filter2, kernel_size=k_size2, activation=act_func,padding='SAME')(cnn3)
cnn3 = MaxPooling1D(pool_size=mp2,padding='SAME')(cnn3)
keras.layers.BatchNormalization(),
cnn3=Dropout(0.5)(cnn3)
#cnn3= GRU (32,dropout=0.5,recurrent_dropout=0.5)(cnn3)
cnn3 = Flatten()(cnn3)
cnn3 = Model(inputs=inputaz, outputs=cnn3)

#---------------yaw--------------------------------------------------------------------

# first input model
cnn4 = Conv1D(filters=filter1, kernel_size=k_size1, activation=act_func,padding='SAME')(inputwx)
cnn4 = MaxPooling1D(pool_size=mp1)(cnn4)
keras.layers.BatchNormalization(),
cnn4=Dropout(0.5)(cnn4)
cnn4 = Conv1D(filters=filter2, kernel_size=k_size2, activation=act_func,padding='SAME')(cnn4)
cnn4 = MaxPooling1D(pool_size=mp2,padding='SAME')(cnn4)
keras.layers.BatchNormalization(),
cnn4=Dropout(0.5)(cnn4)
#cnn4= GRU (32,dropout=0.5,recurrent_dropout=0.5)(cnn4)
cnn4 = Flatten()(cnn4)
cnn4 = Model(inputs=inputwx, outputs=cnn4)



# second input model

cnn5 = Conv1D(filters=filter1, kernel_size=k_size1, activation=act_func,padding='SAME')(inputwy)
cnn5 = MaxPooling1D(pool_size=mp1)(cnn5)
keras.layers.BatchNormalization(),
cnn5=Dropout(0.5)(cnn5)
cnn5 = Conv1D(filters=filter2, kernel_size=k_size2, activation=act_func,padding='SAME')(cnn5)
cnn5 = MaxPooling1D(pool_size=mp2,padding='SAME')(cnn5)
keras.layers.BatchNormalization(),
cnn5=Dropout(0.5)(cnn5)
#cnn5= GRU (32,dropout=0.5,recurrent_dropout=0.5)(cnn5)
cnn5 = Flatten()(cnn5)
cnn5 = Model(inputs=inputwy, outputs=cnn5)



cnn6 = Conv1D(filters=filter1, kernel_size=k_size1, activation=act_func,padding='SAME')(inputwz)
cnn6 = MaxPooling1D(pool_size=mp1)(cnn6)
keras.layers.BatchNormalization(),
cnn6=Dropout(0.5)(cnn6)
cnn6 = Conv1D(filters=filter2, kernel_size=k_size2, activation=act_func,padding='SAME')(cnn6)
cnn6 = MaxPooling1D(pool_size=mp2,padding='SAME')(cnn6)
keras.layers.BatchNormalization(),
cnn6=Dropout(0.5)(cnn6)
#cnn6= GRU (32,dropout=0.5,recurrent_dropout=0.5)(cnn6)
cnn6 = Flatten()(cnn6)
cnn6= Model(inputs=inputwz, outputs=cnn6)


# merge input models
merge1 = concatenate([cnn1.output,cnn2.output,cnn3.output])
merge2 = concatenate([cnn4.output,cnn5.output,cnn6.output])

dense1 = Dense(256, activation=act_func)(merge1)
keras.layers.BatchNormalization(),
dense1=Dropout(0.5)(dense1)
output1 = Dense(1)(dense1)

dense2 = Dense(256, activation=act_func)(merge2)
keras.layers.BatchNormalization(),
dense2=Dropout(0.5)(dense2)
output2 = Dense(1)(dense2)
model= Model(inputs=[cnn1.input,cnn2.input,cnn3.input,cnn4.input,cnn5.input,cnn6.input],outputs=[output1,output2])
'''

In [ ]:
#eğitme işlemi yapar bunu train ederken çalıştır.
'''
optim = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.99, decay=0.01)

model.compile(optimizer=optim,
              loss=['mae','mae'],
              metrics=['mae','mse'])

es = EarlyStopping(monitor='loss',
                   min_delta=1e-15,
                   patience=10,
                   verbose=1)

rlr = ReduceLROnPlateau(monitor='val_loss',
                        factor=0.001,
                        patience=100,
                        verbose=1)

mcp = ModelCheckpoint(filepath="test.h5",
                      monitor='val_loss',
                      verbose=1,
                      save_best_only=True,
                      save_weights_only=False)

tb = TensorBoard('./logs')

print('\t Fitting Data into the Model...')
history = model.fit([X1, X2,X3,X4,X5,X6], [y1,y2],
                    epochs=1000,
                    batch_size=2*n_steps,
                    validation_data=([X1t,X2t,X3t,X4t,X5t,X6t], [yt1,yt2]),
                    verbose=2,
                    shuffle=False,
                    callbacks=[mcp, rlr,es])
'''

In [11]:
#önceden eğitilmiş modeli yükle
from tensorflow.keras.models import load_model

model = load_model('localization.h5')

In [12]:
#norm deltaposition, deltaheading tahmini

yhat = model.predict([X1t, X2t,X3t,X4t,X5t,X6t], verbose=0)
print(yhat)


[array([[7.0982846e-05],
       [7.2367024e-05],
       [7.1018585e-05],
       ...,
       [6.8454654e-05],
       [6.9209142e-05],
       [6.7868852e-05]], dtype=float32), array([[2.9069939e-05],
       [3.4267247e-05],
       [2.6977452e-05],
       ...,
       [3.2075484e-05],
       [3.4349167e-05],
       [3.6691759e-05]], dtype=float32)]


In [13]:
#tahmin ettiğimiz deltaheading ve deltaposition normu ayıralım
yhat=np.asarray(yhat)
yhat.shape
yaw=yhat[1,:,0]
mag=yhat[0,:,0]

In [ ]:
mag

In [ ]:
#tahmin edilen deltaposition normun ilk 5000 datası
arr = np.arange(0,len(mag[0:5000]))

plt.scatter(arr,mag[0:5000],color='red')



In [ ]:
#target deltaposition ilk 5000 datası
arr = np.arange(0,len(at[0:5000]))
plt.scatter(arr[:],at[0:5000],color='blue')

In [ ]:
#tahmin  edilen delta yaw
arr = np.arange(0,len(yaw))

plt.scatter(arr,yaw,color='red')

In [ ]:
#target deltayaw
arr = np.arange(0,len(out_seqt2))
plt.scatter(arr,out_seqt2,color='blue')

In [ ]:
ccc=0
arr = np.arange(0,len(yaw))
xxrt=np.zeros(len(yaw))

for i in arr:
    xxrt[i]=ccc+yaw[i]
    
    ccc=xxrt[i]
    

plt.scatter(arr[:],xxrt,color='red') #tahmin edilen yaw(heading)
arr1 = np.arange(0,len(yyxtest[:,2]))
plt.scatter(arr1[:],yyxtest[:,2],color='blue')#target  yaw(heading)


In [ ]:
#yaw ve normu kullanarak x,y yi hesaplayıp çizdiriyoruz
arr = np.arange(0,len(xxrt))
sx=np.zeros(len(xxrt))
sy=np.zeros(len(xxrt))

for i in arr:
    sx[i]=mag[i]*math.cos(xxrt[i])
    sy[i]=mag[i]*math.sin(xxrt[i])

ddd=0
vvv=0
arr = np.arange(0,len(sx))
xft=np.zeros((len(sx),2))


for i in arr:
    xft[i,0]=ddd+sx[i]
    xft[i,1]=vvv+sy[i]
    
    ddd=xft[i,0]
    vvv=xft[i,1]

plt.scatter(xft[:,1],xft[:,0],linewidth=2, linestyle="-", c="r") #predicted (y,x grafiği) k

plt.scatter(yyxtest[:,1],yyxtest[:,0],linewidth=2, linestyle="-", c="b") #target(y,x grafiği)

In [ ]:

plt.scatter(xft[:,0],xft[:,1],color='red') 



In [22]:
#from tensorflow.keras.models import load_model
#model.save('localization.h5')  # creates a HDF5 file 'my_model.h5'
